
## 일반 필터와 AI 필터를 비교 해볼 수 있음


In [1]:
import sys
import tensorflow as tf
from PIL import Image  # Pillow module
from model import make_model,SuperResolutionModel

In [2]:
model=make_model(320,240)

Model: "super_resolution_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (1, 320, 240, 8)          224       
                                                                 
 sequential_1 (Sequential)   (1, 320, 240, 8)          584       
                                                                 
 sequential_2 (Sequential)   (1, 640, 480, 3)          219       
                                                                 
Total params: 1,027
Trainable params: 1,027
Non-trainable params: 0
_________________________________________________________________


In [3]:
MAX_PEL_VALUE=255

In [4]:
# jpg이미지를 받으면 열어서 텐서로 바꿔서 리턴
def load_jpg(filename, low_res=False):
    image = Image.open(filename)
    if low_res:
        image_vga = image.resize((240, 320)) 
    else:
        image_vga = image.resize((480, 640))
    image.close()
    
    return tf.convert_to_tensor(image_vga, tf.float32)

In [5]:
## model과 인풋이 들어오면 sr을 한 numpy 배열 보냄
def get_output(model,input):
    out_ = model(input[tf.newaxis])
    out = tf.clip_by_value(out_, 0, MAX_PEL_VALUE)

    return out[0]

In [6]:
# Show Input and Output
def analyze_output(inp, out):
    lr_image =Image.fromarray(tf.cast(inp, tf.uint8).numpy())
    lr_image.show(title='Input')
    interpolation = Image.BILINEAR  # interpolation = Image.BILINEAR
    hr_image = lr_image.resize((480, 640), resample = interpolation)
    hr_image.show(title='BILINEAR filter')
    out_image =Image.fromarray(tf.cast(out, tf.uint8).numpy(),)
    out_image.show(title='AI Super Resolution')


In [7]:
def test(input_file):
    inp_lr = load_jpg(input_file, True)
    out_hr = get_output(model,inp_lr)
    analyze_output(inp_lr,out_hr)

In [8]:
test('test_image2.jpg')